In [1]:
# torchtext
import torchtext
from torchtext import data
from torchtext import datasets

# pytorch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch

import numpy as np

# その他もろもろ
from sklearn.model_selection import train_test_split

import MeCab
import gensim

import socket

In [2]:
me = MeCab.Tagger ("-Owakati")
def mecab_tokenizer(text):
    return me.parse(text).split()

In [3]:
embedding = gensim.models.KeyedVectors.load_word2vec_format("model.vec")

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = torch.device("cpu")

BATCH_SIZE = 100 # バッチサイズ
EMBEDDING_DIM = 300 # 単語の埋め込み次元数
LSTM_DIM = 128 # LSTMの隠れ層の次元数
#VOCAB_SIZE = len(embedding.index2word) # 全単語数
TAG_SIZE = 2 # 今回はネガポジ判定を行うのでネットワークの最後のサイズは2
DA = 128 # AttentionをNeural Networkで計算する際の重み行列のサイズ
R = 1 # Attentionの数

In [5]:
class BiLSTMEncoder(nn.Module):
    def __init__(self, embedding_dim, lstm_dim):
        super(BiLSTMEncoder, self).__init__() #親の__init__で一回初期化する
        self.lstm_dim = lstm_dim
        
        # bidirectional=Trueでお手軽に双方向のLSTMにできる
        self.bilstm = nn.LSTM(embedding_dim, lstm_dim, batch_first=True, bidirectional=True)

    def forward(self, sentences):
        #print("forward")
        #分かち書きテキストが来るのでembeddingにする
        embeds = []
        for s in sentences:
            #print(s)
            vecs = []
            for w in s:
                if w in embedding:
                    vecs.append(embedding[w])
                else:
                    vecs.append(np.zeros(EMBEDDING_DIM))
            #vecs = torch.tensor(vecs)
            #print(vecs.shape)
            embeds.append(vecs)
            
        embeds = torch.tensor(embeds).float()
        #print("embeddings shape {}".format(embeds.shape))
        
        # 各隠れ層のベクトルがほしいので第１戻り値を受け取る
        out, _ = self.bilstm(embeds)

        # 前方向と後ろ方向の各隠れ層のベクトルを結合したままの状態で返す
        return out

In [6]:
class SelfAttention(nn.Module):
  def __init__(self, lstm_dim, da, r):
    super(SelfAttention, self).__init__()
    self.lstm_dim = lstm_dim
    self.da = da
    self.r = r
    self.main = nn.Sequential(
        # Bidirectionalなので各隠れ層のベクトルの次元は２倍のサイズになってます。
        nn.Linear(lstm_dim * 2, da), 
        nn.Tanh(),
        nn.Linear(da, r)
    )
  def forward(self, out):
    return F.softmax(self.main(out), dim=1)

In [7]:
class SelfAttentionClassifier(nn.Module):
  def __init__(self, lstm_dim, da, r, tagset_size):
    super(SelfAttentionClassifier, self).__init__()
    self.lstm_dim = lstm_dim
    self.r = r
    self.attn = SelfAttention(lstm_dim, da, r)
    self.main = nn.Linear(lstm_dim * 2 * r, tagset_size)

  def forward(self, out):
    attention_weight = self.attn(out)
    
    feats = torch.tensor([], device=device)
    for i in range(self.r):
        m1 = (out * attention_weight[:,:,i].unsqueeze(2)).sum(dim=1)
        feats = torch.cat([feats, m1], dim = 1)
    #m1 = (out * attention_weight[:,:,0].unsqueeze(2)).sum(dim=1)
    #m2 = (out * attention_weight[:,:,1].unsqueeze(2)).sum(dim=1)
    #m3 = (out * attention_weight[:,:,2].unsqueeze(2)).sum(dim=1)
    #feats = torch.cat([m1, m2, m3], dim=1)
    return F.log_softmax(self.main(feats)), attention_weight

In [11]:
encoder = BiLSTMEncoder(EMBEDDING_DIM, LSTM_DIM).to(device)
encoder.load_state_dict(torch.load("encoder.trained"))
classifier = SelfAttentionClassifier(LSTM_DIM, DA, R, TAG_SIZE).to(device)
classifier.load_state_dict(torch.load("classifier.trained"))

<All keys matched successfully>

In [17]:
text = input()
text_tensor = np.array([mecab_tokenizer(text)])
print(text_tensor)

id2ans = {1: 'positive', 0:'negative'}

encoder_outputs = encoder(text_tensor)
output, attn = classifier(encoder_outputs)
pred = output.data.max(1, keepdim=True)[1]

print(output[0])
#print("{:.02f}".format(np.power(np.e , float(output[0][0]))))
print("再生回数：{:.0f}回".format(-300 * np.power(float(output[0][0]),3) ))
print(pred)
print(id2ans[int(pred)])

classifier
[['classifier']]
tensor([-0.7194, -0.6676], grad_fn=<SelectBackward>)
再生回数：112回
tensor([[1]])
positive


<ipython-input-7-1f20314e62cf>:20: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(self.main(feats)), attention_weight


In [ ]:
with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    # IPアドレスとポートを指定
    s.bind(('localhost', 50007))
    # 1 接続
    s.listen(1)
    # connection するまで待つ
    while True:
        # 誰かがアクセスしてきたら、コネクションとアドレスを入れる
        conn, addr = s.accept()
        with conn:
            while True:
                # データを受け取る
                data = conn.recv(1024)
                if not data:
                    break
                print('data : {}, addr: {}'.format(data, addr))
                # クライアントにデータを返す(b -> byte でないといけない)
                conn.sendall(b'Received: ' + data)